## Drone follow me using Kalman Filters

## Task 1: Video Library

In [ ]:
# !conda install -c conda-forge pytube -y

In [ ]:
from pytube import YouTube
import numpy as np
import cv2
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd

We use Pytube python library for downloading the videos from YouTube. Pytube is a small, dependency-free Python module for accessing videos from the internet. In this use-case, we only download the videos and not the captions as we do not seem to use captions in this assignment.

Official Docs: https://pytube.io/en/latest/user/quickstart.html

In [ ]:
# write function to build the YouTube module's object by supplying the URL in parameter
def Download(link, output_path, filename):
    youtubeObject = YouTube(link)
    youtubeObject = youtubeObject.streams.get_highest_resolution() # obtain video's highest resolution
    try:
        youtubeObject.download(output_path = output_path,
                               filename = filename) # download at specified output path and filename
    except:
        print("An error has occurred")
    print("Download is completed successfully")

In [ ]:
# Download Video 1:
link = input("Enter the YouTube video URL: ") # input the link and press enter
Download(link, output_path = 'inputs/', filename = 'video1.mp4') # begin downloading once we hit 'enter'

Enter the YouTube video URL:  https://www.youtube.com/watch?v=WeF4wpw7w9k


Download is completed successfully


In [ ]:
# Download Video 2:
link = input("Enter the YouTube video URL: ")
Download(link, output_path = 'inputs/', filename = 'video2.mp4')

Enter the YouTube video URL:  https://www.youtube.com/watch?v=2NFwY15tRtA


Download is completed successfully


In [ ]:
# Download Video 3:
link = input("Enter the YouTube video URL: ")
Download(link, output_path = 'inputs/', filename = 'video3.mp4')

Enter the YouTube video URL:  https://www.youtube.com/watch?v=5dRramZVu2Q


Download is completed successfully


## Task 2: Object Detection

We use the Ultralytics API to instantiate a YOLOv8m (medium size version of YOLOv8) object detection model, pretrained on COCO dataset.

For small objects since we use drone imaging, lower-stride models typically fare better as these models generally maintain more detail from the input image, which can be essential for detecting and correctly classifying small objects. Hence we chose to use YOLOv8m model for our purposes. This medium size model also does not have extremely large number of parameters, to run inference using local system.

In [ ]:
from ultralytics import YOLO

In [ ]:
# Load a COCO-pretrained YOLOv8m model
model = YOLO('yolov8m.pt')

# Display model information
model.info()

YOLOv8m summary: 295 layers, 25902640 parameters, 0 gradients, 79.3 GFLOPs


(295, 25902640, 0, 79.3204224)

#### Visualize predictions on local computer

The following code utilizes YOLOv8 pre-trained on COCO dataset for object tracking on a specified input video stream.

The object detection tracking results are visualized by annotating the frame with bounding boxes, labels and confidence scores for detected objects. The annotated frame is then displayed in a separate window.

In [ ]:
import cv2
from ultralytics import YOLO


# Open the video file
video_path = "inputs/video1.mp4"
cap = cv2.VideoCapture(video_path)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


0: 352x640 (no detections), 129.3ms
Speed: 3.4ms preprocess, 129.3ms inference, 5.1ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 142.3ms
Speed: 0.8ms preprocess, 142.3ms inference, 0.5ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 137.0ms
Speed: 0.9ms preprocess, 137.0ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 motor, 139.4ms
Speed: 1.3ms preprocess, 139.4ms inference, 2.8ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 128.7ms
Speed: 0.9ms preprocess, 128.7ms inference, 0.2ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 122.0ms
Speed: 0.9ms preprocess, 122.0ms inference, 0.1ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 126.3ms
Speed: 0.9ms preprocess, 126.3ms inference, 0.2ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 pedestrian, 131.1ms
Speed: 1.0ms preprocess, 131.

#### Save the Video file with predictions using YOLOv8 (medium) pre-trained on COCO dataset

We now save the result of object detection on our three input videos using the YOLOv8 model pre-trained on COCO, we use this model as the baseline. We make further improvement to this object detection model in the next sub-section.

In [ ]:
results = model.track(source='inputs/video1.mp4', persist=True, save=True)
# setting stream=True, will not accumulate inference results in RAM



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1347) /Users/rishienandhan/Drone/inputs/video1.mp4: 352x640 (no detections), 146.3ms
video 1/1 (frame 2/1347) /Users/rishienandhan/Drone/inputs/video1.mp4: 352x640 (no detections), 130.8ms
video 1/1 (frame 3/1347) /Users/rishienandhan/Drone/inputs/video1.mp4: 352x640 (no detections), 126.2ms
video 1/1 (frame 4/1347) /Users/rishienandhan/Drone/inputs/video1.mp4: 352x640 (no detections), 121.4ms
video 1/1 (frame 5/1347) /Users/rishiena

In [ ]:
model = YOLO('yolov8m.pt') # re-initialize model to reset tracking for for handling multiple videos sequentially
results = model.track(source='inputs/video2.mp4', persist=True, save=True)




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/490) /Users/rishienandhan/Drone/inputs/video2.mp4: 352x640 1 person, 1 train, 141.4ms
video 1/1 (frame 2/490) /Users/rishienandhan/Drone/inputs/video2.mp4: 352x640 1 person, 1 boat, 131.8ms
video 1/1 (frame 3/490) /Users/rishienandhan/Drone/inputs/video2.mp4: 352x640 1 person, 128.8ms
video 1/1 (frame 4/490) /Users/rishienandhan/Drone/inputs/video2.mp4: 352x640 1 person, 120.8ms
video 1/1 (frame 5/490) /Users/rishienandhan/Drone/inpu

In [ ]:
model = YOLO('yolov8m.pt') # re-initialize model to reset tracking for for handling multiple videos sequentially
results = model.track(source='/Users/rishienandhan/Drone/inputs/video3.mp4', persist=True, save=True)




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/2250) /Users/rishienandhan/Drone/inputs/video3.mp4: 384x640 4 persons, 2 trucks, 147.9ms
video 1/1 (frame 2/2250) /Users/rishienandhan/Drone/inputs/video3.mp4: 384x640 1 person, 1 motorcycle, 2 trucks, 138.6ms
video 1/1 (frame 3/2250) /Users/rishienandhan/Drone/inputs/video3.mp4: 384x640 3 persons, 1 motorcycle, 2 trucks, 136.9ms
video 1/1 (frame 4/2250) /Users/rishienandhan/Drone/inputs/video3.mp4: 384x640 2 persons, 1 motorcycle, 2

We saved the videos with predictions using YOLOv8m (medium size) pretrained on COCO dataset to 'outputs/COCO_pretrain/' folder to a shared drive which can be accessed using this link: https://drive.google.com/drive/folders/1Bd1D3jyy9EzcCr2YJHxMZ4rRgvGimuCh?usp=sharing.

Notice how in video1.mp4, there are multiple erroneous detections such as cake, boat, knife, train, clock, along with irregular detection of cars, being our object of interest. Also, notice how the detection of the car in video1 is fluctuating across frames; this can be attributed to the object being extremely small in the Drone imagery and hence the object detector fails to constantly pick it up across frames. The same pattern can be observed in output videos video2.mp4, video3.mp4 as well.

So, as an imporvement to our object detection model, we now train it on the VisDrone dataset.


#### Train YOLOv8m on VisDrone dataset

[Bonus Points Question] We now train our YOLOv8m model on VisDrone dataset, which contains objects from 10 classes inlcuding bicycle, pedestrians, car, truck, etc. There are multiple detections across 6471 images that we use for training, 548 images for validation, and 1610 images for test purposes. Each image contains multiple detections across various objects from 10 classes. 

The training part of this pipeline was performed in Colab Pro using V100 GPU, using an image size of 640x640 for 100 epochs. The total training took ~8hours using this GPU. We have used a YAML file to define the dataset configuration. It contains information about the dataset's paths, classes, and other relevant information. The VisDrone.yaml file present in the parent directory is maintained for this purpose. We use the help of the Ultralytics API for training our model. 

The training results across 100 epochs are present in the cell outputs of below cells.


In [ ]:
# Re-load the COCO-pretrained YOLOv8m model
model = YOLO('yolov8m.pt')

# Display model information
model.info()

100%|██████████| 49.7M/49.7M [00:00<00:00, 293MB/s]


YOLOv8m summary: 295 layers, 25902640 parameters, 0 gradients, 79.3 GFLOPs


(295, 25902640, 0, 79.3204224)

In [ ]:
# Train the model on the VisDrone dataset for 10 epochs
results = model.train(data='VisDrone.yaml', epochs=100, imgsz=640,
                      device=0, verbose=True, plots=True, batch=-1,
                      project='/content/drive/My Drive/Drone')


Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=VisDrone.yaml, epochs=100, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=/content/drive/My Drive/Drone, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True,

100%|██████████| 6.23M/6.23M [00:00<00:00, 192MB/s]


AMP: checks passed ✅
AutoBatch: Computing optimal batch size for imgsz=640
AutoBatch: CUDA:0 (Tesla V100-SXM2-16GB) 15.77G total, 0.30G reserved, 0.23G allocated, 15.24G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    25862110       79.09         0.810         33.79         120.1        (1, 3, 640, 640)                    list
    25862110       158.2         1.229         28.06         61.66        (2, 3, 640, 640)                    list
    25862110       316.4         1.829         29.77         61.94        (4, 3, 640, 640)                    list
    25862110       632.8         3.142         33.75         74.93        (8, 3, 640, 640)                    list
    25862110        1266         6.463         55.22         89.94       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 23 for CUDA:0 9.50G/15.77G (60%) ✅


train: Scanning /content/datasets/VisDrone/VisDrone2019-DET-train/labels.cache... 6471 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6471/6471 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/datasets/VisDrone/VisDrone2019-DET-train/images/0000137_02220_d_0000163.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/datasets/VisDrone/VisDrone2019-DET-train/images/0000140_00118_d_0000002.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/datasets/VisDrone/VisDrone2019-DET-train/images/9999945_00000_d_0000114.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/datasets/VisDrone/VisDrone2019-DET-train/images/9999987_00000_d_0000049.jpg: 1 duplicate labels removed


...

                   all        548      38759       0.55      0.419      0.433      0.262

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      13.1G     0.9777     0.5221     0.8514        333        640: 100%|██████████| 282/282 [04:37<00:00,  1.02it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:12<00:00,  1.05s/it]


                   all        548      38759      0.552       0.42      0.434      0.262

100 epochs completed in 9.061 hours.
Optimizer stripped from /content/drive/My Drive/Drone/train/weights/last.pt, 52.0MB
Optimizer stripped from /content/drive/My Drive/Drone/train/weights/best.pt, 52.0MB

Validating /content/drive/My Drive/Drone/train/weights/best.pt...
Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Model summary (fused): 218 layers, 25845550 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:53<00:00,  4.44s/it]


                   all        548      38759      0.543      0.427      0.445      0.271
            pedestrian        548       8844      0.579      0.446      0.485       0.23
                people        548       5125      0.627      0.327      0.384      0.154
               bicycle        548       1287      0.335      0.209      0.184     0.0834
                   car        548      14064      0.771      0.785      0.818      0.595
                   van        548       1975      0.526       0.48      0.474      0.339
                 truck        548        750      0.517      0.393      0.417      0.286
              tricycle        548       1045      0.445      0.352      0.348      0.197
       awning-tricycle        548        532      0.307      0.188       0.18      0.113
                   bus        548        251      0.761      0.594      0.654       0.48
                 motor        548       4886      0.565      0.493      0.503      0.235
Speed: 0.2ms preproce

The 'VisDrone_train_plots' folder in the shared drive contains the training results such as confusion matrix, loss and acuracy plots. The results.csv file in this folder contains various training metrics such as box loss, classification loss, precision, recall, and other metrics for train and validation datasets for every epoch. 

Now, we use the YOLOv8m model pre-trained on COCO and trained on VisDrone dataset to run predictions on our input video streams. 

In [ ]:
# Import trained model on VisDrone:

model = YOLO('/Users/rishienandhan/Drone/runs_VisDrone/train/weights/best.pt')  # load a custom trained model

# Display model information
model.info()

Model summary: 295 layers, 25862110 parameters, 0 gradients, 79.1 GFLOPs


(295, 25862110, 0, 79.0946304)

#### Save the Video file with predictions using YOLOv8 (medium) pre-trained on COCO dataset and trained on VisDrone for 100 epochs

In [ ]:
model = YOLO('/Users/rishienandhan/Drone/runs_VisDrone/train/weights/best.pt') # re-initialize model to reset tracking IDs for for handling multiple videos sequentially
results = model.track(source='inputs/video1.mp4', persist=True, save=True)
# setting stream=True, will not accumulate inference results in RAM



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1347) /Users/rishienandhan/Drone/inputs/video1.mp4: 352x640 (no detections), 134.3ms
video 1/1 (frame 2/1347) /Users/rishienandhan/Drone/inputs/video1.mp4: 352x640 (no detections), 137.5ms
video 1/1 (frame 3/1347) /Users/rishienandhan/Drone/inputs/video1.mp4: 352x640 (no detections), 135.5ms
video 1/1 (frame 4/1347) /Users/rishienandhan/Drone/inputs/video1.mp4: 352x640 1 motor, 113.9ms
video 1/1 (frame 5/1347) /Users/rishienandhan/Dr

In [ ]:
model = YOLO('/Users/rishienandhan/Drone/runs_VisDrone/train/weights/best.pt') # re-initialize model to reset tracking IDs for for handling multiple videos sequentially
results = model.track(source='inputs/video2.mp4', persist=True, save=True)




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/490) /Users/rishienandhan/Drone/inputs/video2.mp4: 352x640 1 pedestrian, 1 motor, 121.3ms
video 1/1 (frame 2/490) /Users/rishienandhan/Drone/inputs/video2.mp4: 352x640 1 pedestrian, 1 motor, 114.4ms
video 1/1 (frame 3/490) /Users/rishienandhan/Drone/inputs/video2.mp4: 352x640 1 pedestrian, 111.4ms
video 1/1 (frame 4/490) /Users/rishienandhan/Drone/inputs/video2.mp4: 352x640 1 pedestrian, 116.7ms
video 1/1 (frame 5/490) /Users/rishien

In [ ]:
model = YOLO('/Users/rishienandhan/Drone/runs_VisDrone/train/weights/best.pt') # re-initialize model to reset tracking IDs for for handling multiple videos sequentially
results = model.track(source='/Users/rishienandhan/Drone/inputs/video3.mp4', persist=True, save=True)




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/2250) /Users/rishienandhan/Drone/inputs/video3.mp4: 384x640 1 pedestrian, 135.5ms
video 1/1 (frame 2/2250) /Users/rishienandhan/Drone/inputs/video3.mp4: 384x640 1 pedestrian, 132.4ms
video 1/1 (frame 3/2250) /Users/rishienandhan/Drone/inputs/video3.mp4: 384x640 1 pedestrian, 130.2ms
video 1/1 (frame 4/2250) /Users/rishienandhan/Drone/inputs/video3.mp4: 384x640 1 pedestrian, 125.8ms
video 1/1 (frame 5/2250) /Users/rishienandhan/Drone/

We now save the videos with predictions using YOLOv8m (medium size) trained on VisDrone to 'outputs/VisDrone_train_results/' folder in the shared drive which can be accessed using this link: https://drive.google.com/drive/folders/1Bd1D3jyy9EzcCr2YJHxMZ4rRgvGimuCh?usp=sharing.

With our newly trained model, we are able to avoid erroneous detections and more importantly, pick up important features (car and people) more constantly across our video frames. You can observe in video1.mp4 that the detection of the car is more stable and is being picked up constantly across frames, which would certainly help us in tracking in next step. 

The same pattern is observed in other output videos as well. Hence, training our model on VisDrone dataset certainly helps in a more stable and continuous object tracking. 


## Task 3: Kalman Filters

The following function creates and initializes a Kalman Filter model along with its parameters for 2D motion object tracking model with specified dimensions.

We can then integrate this Kalman Filter model into the object tracking pipeline along with the object detections obtained from Task 2 to track the pedestrians and vehicles in the videos. We can use the predicted object positions from the Kalman Filter to plot trajectories and improve tracking accuracy. We plan to detect and track cars and people across video frames. 

In [ ]:
# !conda install -c conda-forge filterpy -y

In [ ]:
import cv2
from filterpy.kalman import KalmanFilter
import numpy as np

# Define Kalman Filter model for 2D object tracking
def create_kalman_filter(dim_x, dim_z):
    kf = KalmanFilter(dim_x=dim_x, dim_z=dim_z)

    # Define state transition matrix (motion model) for 2D tracking
    kf.F = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]])

    # Define measurement function (observation model) for 2D tracking
    kf.H = np.array([[1, 0, 0, 0], [0, 1, 0, 0]])

    # Define process noise covariance matrix
    kf.Q *= 0.01

    # Define measurement noise covariance matrix
    kf.R *= 0.1

    # Initialize state vector and covariance matrix for 2D tracking
    kf.x = np.array([[0], [0], [0], [0]])
    kf.P = np.eye(dim_x) * 100

    return kf

In the following cell, we integrate the Kalman Filter into our object detection pipeline to estimate state of objects in subsequent frames.
Following are the steps we follow:

- Initialize the YOLO model with pre-trained weights from VisDrone dataset training.
- Create dictionaries to store Kalman filters (`kalman_filters`) for each object ID and past trajectories (`trajectories`) for each object ID.
- Define the number of past frames to keep track of (`num_past_frames`).
- Enter a loop to process each frame of the video until the end of the video is reached or the user presses 'q' to exit.
- Perform object detection and tracking on the frame using the YOLO model.
- Extract bounding boxes, object IDs, and predicted classes from the detection results.
- Iterate over each detected object in a given frame and update its Kalman filter to predict its next position (St+1).
- Store the current position in the trajectory list of past estimates for each object to draw trajectory.
- Visualize the object trajectory and bounding box on the frame by drawing lines and rectangles.
- Annotate the frame with the object class and ID using `cv2.putText`.
- Terminate the loop and release the video capture object when the 'q' key is pressed.

The inline comments in the code provide additional details.

In [ ]:

# Import pretrained model on VisDrone
model = YOLO('/Users/rishienandhan/Drone/runs_VisDrone/train/weights/best.pt')

# Open the video file
video_path = "inputs/video3.mp4"
cap = cv2.VideoCapture(video_path)

# Initialize dictionary to store Kalman Filters for each object ID
kalman_filters = {}

# Initialize dictionary to store past trajectories for each object ID
trajectories = {}

# Define the number of past frames to keep track of
num_past_frames = 10

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection using YOLOv8 pretrained on COCO and trained on VisDrone
    results = model.track(frame, persist=True)

    # Iterate over each detection result
    for result in results:
        # Extract bounding boxes, object IDs and predicted classes from the result
        detections = result.boxes.xyxy # xyxy format
        object_ids = result.boxes.id
        classes = result.boxes.cls

        # Check if there are any detections before iterating
        if detections is not None and object_ids is not None:
            # For each detected object, update Kalman Filter prediction
            for detection, object_id, obj_class in zip(detections, object_ids, classes):
                x1, y1, x2, y2 = detection  # Bounding box coordinates
                center_x = (x1 + x2) / 2  # Calculating center x
                center_y = (y1 + y2) / 2  # Calculating center y

                # If object_id not in Kalman Filters dictionary, initialize a new Kalman Filter
                if object_id not in kalman_filters:
                    kalman_filters[object_id] = create_kalman_filter(dim_x=4, dim_z=2)
                    # Initialize Kalman Filter state using the detected object position
                    kalman_filters[object_id].x[:2] = np.array([[center_x], [center_y]])

                # Predict the next state (position and velocity) using Kalman Filter
                kalman_filters[object_id].predict()

                # Update the measurement (observed position) from the detection
                measurement = np.array([[center_x], [center_y]])

                # Update the Kalman Filter with the measurement
                kalman_filters[object_id].update(measurement)

                # Get the updated position estimate from Kalman Filter
                estimated_position = kalman_filters[object_id].x[:2]

                # Plot the object trajectory using the estimated position
                cv2.circle(frame, (int(estimated_position[0]), int(estimated_position[1])), 5, (0, 255, 0), -1) # green color chosen

                # Draw bounding box around the detected object
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)

                # PLOT TRAJECTORY OF PAST ESTIMATES:
                # Store the current position in the trajectory list of past estimates
                if int(object_id) not in trajectories:
                    trajectories[int(object_id)] = [] # if new object, maintain separate new list
                # if current object already seen before, append position to list
                trajectories[int(object_id)].append((int(estimated_position[0]), int(estimated_position[1])))

                # Limit the length of past trajectory list to keep track of only a few past frames
                if len(trajectories[int(object_id)]) > num_past_frames:
                    trajectories[int(object_id)].pop(0)

                # Plot the trajectory using the stored positions of past estimates
                for index, item in enumerate(trajectories[int(object_id)]):
                    if len(trajectories[int(object_id)]) < 1:
                        break
                    cv2.line(frame, item, trajectories[int(object_id)][index], [255, 0, 0], 2)

                # Draw object class and ID on the frame
                cv2.putText(frame, f'{result.names[int(obj_class)]}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)


    # Display the frame with object trajectories and bounding boxes
    cv2.imshow('Frame', frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()



0: 384x640 1 pedestrian, 130.8ms
Speed: 1.2ms preprocess, 130.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)



/var/folders/7f/xmsdl1017zl1651xcmrx6y9r0000gn/T/ipykernel_2124/3096873428.py:59: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.circle(frame, (int(estimated_position[0]), int(estimated_position[1])), 5, (0, 255, 0), -1) # green color chosen
/var/folders/7f/xmsdl1017zl1651xcmrx6y9r0000gn/T/ipykernel_2124/3096873428.py:69: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  trajectories[int(object_id)].append((int(estimated_position[0]), int(estimated_position[1])))


0: 384x640 1 pedestrian, 318.5ms
Speed: 20.0ms preprocess, 318.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pedestrian, 168.5ms
Speed: 0.8ms preprocess, 168.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pedestrian, 175.4ms
Speed: 1.1ms preprocess, 175.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pedestrian, 148.2ms
Speed: 0.7ms preprocess, 148.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pedestrian, 148.5ms
Speed: 0.8ms preprocess, 148.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pedestrian, 162.9ms
Speed: 0.7ms preprocess, 162.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 pedestrians, 161.8ms
Speed: 0.8ms preprocess, 161.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 pedestrians, 139.1ms
Speed: 0.8ms preprocess, 139.1ms inferen

#### Save the predictions as a video file:

We encompass the above code as a function to output the predictions as a video file along with bounding boxes, class labels, state estimates, and trajectory.

In [ ]:
def track_objects_outputVideo(video_path, output_filename):
    # Import pretrained model on VisDrone
    model = YOLO('/Users/rishienandhan/Drone/runs_VisDrone/train/weights/best.pt')

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get the video frame dimensions
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Define the codec and create VideoWriter object
    out = cv2.VideoWriter(output_filename, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    # Initialize dictionary to store Kalman Filters for each object ID
    kalman_filters = {}

    # Initialize dictionary to store past trajectories for each object ID
    trajectories = {}

    # Define the number of past frames to keep track of
    num_past_frames = 10

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Perform object detection using YOLOv8 pretrained on COCO and trained on VisDrone
        results = model.track(frame, persist=True)

        # Iterate over each detection result
        for result in results:
            # Extract bounding boxes, object IDs and predicted classes from the result
            detections = result.boxes.xyxy # xyxy format
            object_ids = result.boxes.id
            classes = result.boxes.cls

            # Check if there are any detections before iterating
            if detections is not None and object_ids is not None:
                # For each detected object, update Kalman Filter prediction
                for detection, object_id, obj_class in zip(detections, object_ids, classes):
                    x1, y1, x2, y2 = detection  # Bounding box coordinates
                    center_x = (x1 + x2) / 2  # Calculating center x
                    center_y = (y1 + y2) / 2  # Calculating center y

                    # If object_id not in Kalman Filters dictionary, initialize a new Kalman Filter
                    if object_id not in kalman_filters:
                        kalman_filters[object_id] = create_kalman_filter(dim_x=4, dim_z=2)
                        # Initialize Kalman Filter state using the detected object position
                        kalman_filters[object_id].x[:2] = np.array([[center_x], [center_y]])

                    # Predict the next state (position and velocity) using Kalman Filter
                    kalman_filters[object_id].predict()

                    # Update the measurement (observed position) from the detection
                    measurement = np.array([[center_x], [center_y]])

                    # Update the Kalman Filter with the measurement
                    kalman_filters[object_id].update(measurement)

                    # Get the updated position estimate from Kalman Filter
                    estimated_position = kalman_filters[object_id].x[:2]

                    # Plot the object trajectory using the estimated position
                    cv2.circle(frame, (int(estimated_position[0]), int(estimated_position[1])), 5, (0, 255, 0), -1) # green color chosen

                    # Draw bounding box around the detected object
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)

                    # PLOT TRAJECTORY OF PAST ESTIMATES:
                    # Store the current position in the trajectory list of past estimates
                    if int(object_id) not in trajectories:
                        trajectories[int(object_id)] = [] # if new object, maintain separate new list
                    # if current object already seen before, append position to list
                    trajectories[int(object_id)].append((int(estimated_position[0]), int(estimated_position[1])))

                    # Limit the length of past trajectory list to keep track of only a few past frames
                    if len(trajectories[int(object_id)]) > num_past_frames:
                        trajectories[int(object_id)].pop(0)

                    # Plot the trajectory using the stored positions of past estimates
                    for index, item in enumerate(trajectories[int(object_id)]):
                        if len(trajectories[int(object_id)]) < 1:
                            break
                        cv2.line(frame, item, trajectories[int(object_id)][index], [255, 0, 0], 2)

                    # Draw object class and ID on the frame
                    cv2.putText(frame, f'{result.names[int(obj_class)]}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Write the frame to the output video file
        out.write(frame)


    # Release video capture and close windows
    cap.release()
    out.release()
    cv2.destroyAllWindows()


In [ ]:
# Call the function to output predictions for video1.mp4
video_path = "inputs/video1.mp4"
output_filename = "outputs/Final_predictions/video1_op.mp4"
track_objects_outputVideo(video_path, output_filename)


0: 352x640 (no detections), 144.7ms
Speed: 1.3ms preprocess, 144.7ms inference, 0.2ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 136.2ms
Speed: 0.7ms preprocess, 136.2ms inference, 0.2ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 131.2ms
Speed: 0.9ms preprocess, 131.2ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 motor, 130.3ms
Speed: 1.3ms preprocess, 130.3ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 145.6ms
Speed: 0.8ms preprocess, 145.6ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 133.4ms
Speed: 1.0ms preprocess, 133.4ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 134.8ms
Speed: 1.8ms preprocess, 134.8ms inference, 0.2ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 pedestrian, 139.3ms
Speed: 0.9ms preprocess, 139.

/var/folders/7f/xmsdl1017zl1651xcmrx6y9r0000gn/T/ipykernel_2124/3916753545.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.circle(frame, (int(estimated_position[0]), int(estimated_position[1])), 5, (0, 255, 0), -1) # green color chosen
/var/folders/7f/xmsdl1017zl1651xcmrx6y9r0000gn/T/ipykernel_2124/3916753545.py:77: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  trajectories[int(object_id)].append((int(estimated_position[0]), int(estimated_position[1])))


0: 352x640 1 car, 320.8ms
Speed: 18.0ms preprocess, 320.8ms inference, 0.4ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 car, 137.3ms
Speed: 1.0ms preprocess, 137.3ms inference, 0.5ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 car, 138.3ms
Speed: 0.6ms preprocess, 138.3ms inference, 0.4ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 car, 135.3ms
Speed: 0.9ms preprocess, 135.3ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 car, 141.8ms
Speed: 0.7ms preprocess, 141.8ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 car, 150.0ms
Speed: 0.7ms preprocess, 150.0ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 car, 137.6ms
Speed: 0.7ms preprocess, 137.6ms inference, 0.5ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 car, 131.8ms
Speed: 0.9ms preprocess, 131.8ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)


In [ ]:
# Call the function to output predictions for video2.mp4
video_path = "inputs/video2.mp4"
output_filename = "outputs/Final_predictions/video2_op.mp4"
track_objects_outputVideo(video_path, output_filename)


0: 352x640 1 pedestrian, 1 motor, 138.1ms
Speed: 1.2ms preprocess, 138.1ms inference, 0.8ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 pedestrian, 1 motor, 132.4ms
Speed: 0.8ms preprocess, 132.4ms inference, 0.5ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 pedestrian, 122.6ms
Speed: 0.7ms preprocess, 122.6ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 pedestrian, 128.1ms
Speed: 0.8ms preprocess, 128.1ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 pedestrian, 139.1ms
Speed: 0.9ms preprocess, 139.1ms inference, 0.5ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 pedestrian, 129.7ms
Speed: 0.9ms preprocess, 129.7ms inference, 0.7ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 pedestrian, 125.4ms
Speed: 0.9ms preprocess, 125.4ms inference, 0.4ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 pedestrian, 133.5ms
Speed: 0.8ms preprocess,

/var/folders/7f/xmsdl1017zl1651xcmrx6y9r0000gn/T/ipykernel_2124/3916753545.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.circle(frame, (int(estimated_position[0]), int(estimated_position[1])), 5, (0, 255, 0), -1) # green color chosen
/var/folders/7f/xmsdl1017zl1651xcmrx6y9r0000gn/T/ipykernel_2124/3916753545.py:77: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  trajectories[int(object_id)].append((int(estimated_position[0]), int(estimated_position[1])))


0: 352x640 1 pedestrian, 320.0ms
Speed: 10.7ms preprocess, 320.0ms inference, 0.4ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 pedestrian, 138.8ms
Speed: 0.8ms preprocess, 138.8ms inference, 0.5ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 pedestrian, 138.5ms
Speed: 0.8ms preprocess, 138.5ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 pedestrian, 121.6ms
Speed: 0.6ms preprocess, 121.6ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 pedestrian, 116.4ms
Speed: 0.7ms preprocess, 116.4ms inference, 0.4ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 pedestrian, 120.5ms
Speed: 0.6ms preprocess, 120.5ms inference, 0.6ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 pedestrian, 125.4ms
Speed: 0.7ms preprocess, 125.4ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 pedestrian, 130.7ms
Speed: 0.7ms preprocess, 130.7ms inference

In [ ]:
# Call the function to output predictions for video3.mp4
video_path = "inputs/video3.mp4"
output_filename = "outputs/Final_predictions/video3_op.mp4"
track_objects_outputVideo(video_path, output_filename)


0: 384x640 1 pedestrian, 184.1ms
Speed: 1.0ms preprocess, 184.1ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)



/var/folders/7f/xmsdl1017zl1651xcmrx6y9r0000gn/T/ipykernel_2124/3916753545.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.circle(frame, (int(estimated_position[0]), int(estimated_position[1])), 5, (0, 255, 0), -1) # green color chosen
/var/folders/7f/xmsdl1017zl1651xcmrx6y9r0000gn/T/ipykernel_2124/3916753545.py:77: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  trajectories[int(object_id)].append((int(estimated_position[0]), int(estimated_position[1])))


0: 384x640 1 pedestrian, 352.0ms
Speed: 11.9ms preprocess, 352.0ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pedestrian, 199.1ms
Speed: 0.9ms preprocess, 199.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pedestrian, 191.6ms
Speed: 0.8ms preprocess, 191.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pedestrian, 178.8ms
Speed: 1.0ms preprocess, 178.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pedestrian, 187.8ms
Speed: 0.8ms preprocess, 187.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pedestrian, 163.9ms
Speed: 1.0ms preprocess, 163.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 pedestrians, 152.4ms
Speed: 0.9ms preprocess, 152.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 pedestrians, 126.9ms
Speed: 0.7ms preprocess, 126.9ms inferen

We save the predictions and state estimates as a video file in 'outputs/Final_predictions/' folder in the shared drive which can be accessed using this link: https://drive.google.com/drive/folders/1Bd1D3jyy9EzcCr2YJHxMZ4rRgvGimuCh?usp=sharing.


In each video outputs, the green point is the predicted estimate of the state of object (centroid of detected object) in next frame(St+1). The red rectangle is the bounding box predicted using the object detector and the blue trailing points represents the past state estimates of an object from its past 10 frames, which we call trajectory. 


**NOTE:** Notice how the blue dots represents the past state estimates of an object from its past 10 frames, but it is important to note this is not very indicative of its actual trajectory as the camera also moves relative to the object causing the blue points to be extremely wavery. Since the object follows a motion model and our camera also moves, the trajectory points (blue) may not be realistic.

What is important to note is our green dot (centroid of the object), which gives the estimate of the next state of the object (St+1) in the next time step (next frame). We obtain the estimate of the next immediate state of the object as this green dot, successfully through the recursive state estimate model (RSE) using Kalman Filters.


Each object detected in the video frames have a unique ID maintained, which helps track multiple objects simultaneously in a video frame. There is a unique bounding box and state estimate for every unique object detected across frames. Note from the code that we maintain  dictionary to store Kalman Filters for each object ID, so have one Kalman filter to track each of the required and present objects (cars and people).


To address false positives in object detection, we can use some of the following methods:
1. **Temporal Consistency with Tracking:** The tracker, especially when using methods like Kalman filters, provides temporal consistency by predicting the position of objects in subsequent frames based on their previous positions. By leveraging this temporal information, we can verify the consistency of detections over time. If a detection is inconsistent with the predicted trajectory, it is more likely to be a false positive and can be discarded.

1. Dynamically Adjusting Detection Threshold: Object detection such as YOLOv8 produce detection scores along with bounding boxes. By adjusting the detection threshold for each class individually based on the precision required, we can filter out detections with low confidence scores dynamically.

3. Non-Maximum Suppression (NMS): NMS is a technique used to suppress multiple overlapping bounding boxes for the same object. It retains only the bounding box with the highest confidence score while suppressing others. This helps in removing redundant detections and reducing false positives.
